In [ ]:
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import os

import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.metrics import Precision, Recall
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.preprocessing import image

import matplotlib.pyplot as plt

#### Changes to baseline_v1
- Increase number of filters in deeper state
- Data augmentation

## Load Dataset

In [ ]:
train_dir = './ds/train/'
test_dir = './ds/test/'

classes = [os.path.basename(x[0]) for x in os.walk("./ds/train") if x[0]][1:]
num_classes = len(classes)
print(classes)

In [ ]:
def load_data(path):
    data = []
    for c, category in enumerate(classes):
        images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(path + category) for f in filenames]

        for img_path in images:
            # Note that we apply the same preprocessing and target images of size 224x224
            # in order to match the setup of original VGG16 model
            img = image.load_img(img_path, target_size=(224, 224))  
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x) 

            data.append({'x' : np.array(x[0]), 'y' : c})

    return data


data_train_val = load_data(train_dir)
data_test = load_data(test_dir)

In [ ]:
X_train_val, y_train_val = np.array([t["x"] for t in data_train_val]), [t["y"] for t in data_train_val]
X_test, y_test = np.array([t["x"] for t in data_test]), [t["y"] for t in data_test]

## Data Enhancement

#### Image Augementation

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    # data_format='channels_last',
    preprocessing_function=preprocess_input
)

## Data Analysis

In [ ]:
unique, counts = np.unique(y_train_val, return_counts=True)
ticks = np.arange(len(counts))
plt.figure(figsize=(14,6))
plt.grid(color='w', axis='y')
plt.bar(ticks, counts)
plt.xticks(ticks, classes, rotation=75)
plt.title('Number of images per class')
plt.show()

## Prepare Data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42, test_size=0.2)

# Convert classes to one-hot vectors
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

print('Train images:', len(y_train), y_train.shape)
print('Test images:', len(y_test), y_test.shape)
print('Val images:', len(y_val), y_val.shape)

## Model Creation and Training

In [ ]:
def get_model():
    model = Sequential()
    print("Input dimensions:", X_train.shape[1:])

    model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # model.summary()

    return model


In [ ]:
baseline_model = get_model()

baseline_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', Precision(), Recall()])

In [ ]:
# Create checkpoints during training
filepath = './checkpoints/baseline_v1_{epoch:02d}_{val_accuracy:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = baseline_model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(X_val, y_val),
                    callbacks=callbacks_list)

## Evaluation

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()